# Chains

Hello everyone! Here, we will explore the most important concept in langchain, "chaining" 

Concepts covered in this notebook are:
1. [Basic Chaining](#1)
2. [Chaining under the Hood](#2)
3. [Extended Chaining](#3)
4. [Parallel Chaining](#4)
5. [Branched Chaining](#5)

Prerequisites (I hope you have understood these concepts):
1. language model
2. prompt template
3. output parser

## Load environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv()

## By default, load_dotenv() will assign environment variables into os.environ, like following code:
## See environment variables in .env file
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] =  os.getenv('LANGCHAIN_API_KEY')
# os.environ["OPENAPI_KEY"] =  os.getenv('OPENAI_KEY')

True

Let's create language model instance, prompt template, and output parser

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

model = ChatOpenAI(model="gpt-3.5-turbo")

prompt_template = ChatPromptTemplate([
    ("system", "Translete the following into {language}"),
    ("user", "{text}")
])

parser = StrOutputParser()

# Basic Chaining

"Chaining" is operation to combine sequential processes — for example from prompt templates to output parsers— using the pipe (`|`) operator.

In [8]:
chain = prompt_template | model | parser

chain.invoke({"language": "indonesia", "text": "How are you?"})

'Apa kabar?'

## 2 - Chaining Under the Hood <div id="2"></div>

Chaining is incredible operation. Knowing that it is a sequential processes combination is enough. But if you want to master chaining deeply, you must understand chain under the hood. I will explain it with 3 levels of abstraction.

### 2.1 - Chaining is Passing Output From Previous to Next

What happens is:



- We invoke `chain` object and pass arguments into it (language and text). 
- Then we pass that arguments to `prompt_template` invocation to convert it into messages as output
- Then we pass the messages to `model` invocation to generate model answer
- Then we pass model answer to `parser` invocation to convert it into text

## Chaining Under the Hood

Chaining works by creating a `Chain` object that takes the output of one function as input to the next.

How can tis works?

In LangChain, chaining operations is achieved by using the `Runnable` class and its methods:

1. **`__or__` Method**:
   - Overrides the pipe operation (`|`) to pass output from one function to the next.
   - **Example**:
     ```python
     class Runnable:
         def __or__(self, other):
             return ChainedRunnable(self, other)
     ```

2. **`invoke` Method**:
   - Executes the operation and processes input data, facilitating chaining.
   - **Example**:
     ```python
     class Runnable:
         def invoke(self, input_data):
             # Process input_data
             pass
     ```

### Example

```python
class ChainedRunnable(Runnable):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def invoke(self, input_data):
        return self.right.invoke(self.left.invoke(input_data))
```

By using `Runnable` with these methods, components like `model`, `prompt_template`, and `parser` can be chained, passing the output from one to the next seamlessly.
